<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/Sliceing_feature_engineering(graphcodebert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate
!pip install datasets

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, RobertaConfig, RobertaModel, RobertaPreTrainedModel, Trainer, TrainingArguments
import torch.nn as nn
from sklearn.metrics import f1_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# CUDA 디버깅 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# CSV 파일 로드
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cwe file/sliceing_dataset.csv")

In [5]:
df

,Unnamed: 0,code_x,lang,CWE ID,vul,code_y
0,0,void CWE415_Double_Free__malloc_free_char_01_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...
1,1,void CWE415_Double_Free__malloc_free_char_02_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...
2,2,void CWE415_Double_Free__malloc_free_char_03_b...,c,CWE-415,1,data = NULL;\nif(5==5)\ndata = (char *)malloc(...
3,3,void CWE415_Double_Free__malloc_free_char_04_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...
4,4,void CWE415_Double_Free__malloc_free_char_05_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...
...,...,...,...,...,...,...
7289,9911,"static void fix_dl_name(MEM_ROOT *root, LEX_ST...",c,CWE-416,1,const size_t so_ext_len= sizeof(SO_EXT) - 1;\n...
7290,9912,"static int post_msg(struct pptp_conn_t *conn, ...",c,CWE-787,1,"if (conn->out_size) {\nn=write(conn->hnd.fd, b..."
7291,9913,static void zlib_stateful_free_ex_data(void *o...,c,CWE-399,1,struct zlib_state *state = (struct zlib_state ...
7292,9914,COMP_METHOD *COMP_zlib(void)\n\t{\n\tCOMP_METH...,c,CWE-399,1,COMP_METHOD *meth = &zlib_method_nozlib;


In [6]:
df.rename(columns={'code_x': 'code'}, inplace=True)

In [7]:
cwe_counts = df['CWE ID'].value_counts().reset_index()
cwe_counts.columns = ['CWE ID', 'count']
cwe_counts['labels'] = range(len(cwe_counts))

In [8]:
df = df.merge(cwe_counts[['CWE ID', 'labels']], on='CWE ID', how='left')
df = df.drop(columns = ['Unnamed: 0'])

In [9]:
df

,code,lang,CWE ID,vul,code_y,labels
0,void CWE415_Double_Free__malloc_free_char_01_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...,0
1,void CWE415_Double_Free__malloc_free_char_02_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...,0
2,void CWE415_Double_Free__malloc_free_char_03_b...,c,CWE-415,1,data = NULL;\nif(5==5)\ndata = (char *)malloc(...,0
3,void CWE415_Double_Free__malloc_free_char_04_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...,0
4,void CWE415_Double_Free__malloc_free_char_05_b...,c,CWE-415,1,data = NULL;\ndata = (char *)malloc(100*sizeof...,0
...,...,...,...,...,...,...
7289,"static void fix_dl_name(MEM_ROOT *root, LEX_ST...",c,CWE-416,1,const size_t so_ext_len= sizeof(SO_EXT) - 1;\n...,5
7290,"static int post_msg(struct pptp_conn_t *conn, ...",c,CWE-787,1,"if (conn->out_size) {\nn=write(conn->hnd.fd, b...",4
7291,static void zlib_stateful_free_ex_data(void *o...,c,CWE-399,1,struct zlib_state *state = (struct zlib_state ...,7
7292,COMP_METHOD *COMP_zlib(void)\n\t{\n\tCOMP_METH...,c,CWE-399,1,COMP_METHOD *meth = &zlib_method_nozlib;,7


In [10]:
df_count = df[['CWE ID','labels']].value_counts()
df_count

CWE ID   labels
CWE-415  0         1658
CWE-119  1         1236
CWE-20   2          813
CWE-125  3          808
CWE-787  4          717
CWE-416  5          588
CWE-476  6          519
CWE-399  7          503
CWE-190  8          452
Name: count, dtype: int64

In [11]:
df = df[['code', 'code_y','labels']]

In [12]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
# 오버샘플링 적용
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(df.drop(columns=['labels']), df['labels'])

# 오버샘플링 후 클래스 분포 확인
print(f"Resampled dataset shape: {Counter(y_res)}")

# 오버샘플링된 데이터를 다시 Dataset 객체로 변환

df = pd.concat([X_res, y_res], axis=1)

Resampled dataset shape: Counter({0: 1658, 1: 1658, 2: 1658, 5: 1658, 7: 1658, 8: 1658, 3: 1658, 6: 1658, 4: 1658})


In [13]:
def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    df['code'] = df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    df['code'] = df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    df['code'] = df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    df['code'] = df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    df['code'] = df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    df['code'] = df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'\n'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    df['code'] = df['code'].replace(re.compile(r'return;'), "", regex=True)

    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(file_name, index=False)
    return df  # 수정된 데이터프레임 반환

df = preprocess(df, "preprocess.csv")

df = pd.read_csv("preprocess.csv")

In [14]:
# code와 code_y 컬럼, 라벨 컬럼, 피쳐 컬럼 분리
code_texts = df['code'].tolist()
code_y_texts = df['code_y'].tolist()
labels = df['labels'].tolist()

In [15]:
# 라벨이 0부터 9까지 있는지 확인
assert all(0 <= label < 9 for label in labels), "라벨이 0부터 9까지의 범위에 있어야 합니다."

features = df.drop(columns=['code', 'code_y', 'labels']).values.astype(np.float32)  # float 타입으로 변환

# 피처 크기 확인 및 설정
feature_size = features.shape[1]
print(f"Feature size: {feature_size}")

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

Feature size: 0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
# code와 code_y 컬럼을 토큰화
encodings = tokenizer(code_texts, padding=True, truncation=True, return_tensors='pt')
code_y_encodings = tokenizer(code_y_texts, padding=True, truncation=True, return_tensors='pt')

In [17]:
# 데이터셋 클래스 정의
class CodeDataset(Dataset):
    def __init__(self, encodings, code_y_encodings, labels, features):
        self.encodings = encodings
        self.code_y_encodings = code_y_encodings
        self.labels = labels
        self.features = features

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        code_y_item = {f'code_y_{key}': val[idx] for key, val in self.code_y_encodings.items()}
        item.update(code_y_item)
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        item['features'] = torch.tensor(self.features[idx], dtype=torch.float)
        return item

In [18]:
# 데이터셋 생성 및 분리
train_texts, val_texts, train_code_y_texts, val_code_y_texts, train_labels, val_labels, train_features, val_features = train_test_split(
    code_texts, code_y_texts, labels, features, test_size=0.2, random_state=42)

train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
val_encodings = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')

train_code_y_encodings = tokenizer(train_code_y_texts, padding=True, truncation=True, return_tensors='pt')
val_code_y_encodings = tokenizer(val_code_y_texts, padding=True, truncation=True, return_tensors='pt')

train_dataset = CodeDataset(train_encodings, train_code_y_encodings, train_labels, train_features)
val_dataset = CodeDataset(val_encodings, val_code_y_encodings, val_labels, val_features)


In [19]:
# 모델 정의
class CodeBERTConfig(RobertaConfig):
    model_type = "codebert"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.max_position_embeddings = 514  # 사전 학습된 모델과 일치
        self.type_vocab_size = 1  # 사전 학습된 모델과 일치
        self.output_attentions = True  # 어텐션 가중치 반환 설정

class CodeBERTForVulnerabilityDetectionWithFeatures(RobertaPreTrainedModel):
    config_class = CodeBERTConfig

    def __init__(self, config, feature_size):
        super().__init__(config)
        self.num_labels = 9  # 0부터 9까지의 10개 클래스
        self.feature_size = feature_size
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.feature_dense = nn.Linear(self.feature_size, config.hidden_size)
        self.code_y_dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size * 3, self.num_labels)  # Concatenate RoBERTa output with features and code_y
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, code_y_input_ids=None, code_y_attention_mask=None, features=None, labels=None, return_attentions=False):
        if features is None:
            raise ValueError("features를 제공해야 합니다.")

        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, output_attentions=return_attentions)
        sequence_output = outputs[0]

        code_y_outputs = self.roberta(code_y_input_ids, attention_mask=code_y_attention_mask)
        code_y_sequence_output = code_y_outputs[0]

        pooled_output = sequence_output[:, 0, :]
        pooled_output = self.dropout(pooled_output)

        code_y_pooled_output = code_y_sequence_output[:, 0, :]
        code_y_pooled_output = self.dropout(code_y_pooled_output)
        code_y_pooled_output = self.code_y_dense(code_y_pooled_output)
        code_y_pooled_output = self.dropout(code_y_pooled_output)

        feature_output = self.feature_dense(features)
        feature_output = self.dropout(feature_output)

        combined_output = torch.cat((pooled_output, code_y_pooled_output, feature_output), dim=1)
        logits = self.classifier(combined_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            if return_attentions:
                attentions = outputs.attentions
                return {"loss": loss, "logits": logits, "attentions": attentions}
            return {"loss": loss, "logits": logits}

        if return_attentions:
            attentions = outputs.attentions
            return {"logits": logits, "attentions": attentions}

        return {"logits": logits}

In [20]:
# 커스텀 메트릭스 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics

In [21]:
# 모델 초기화
config = CodeBERTConfig()
model = CodeBERTForVulnerabilityDetectionWithFeatures(config, feature_size=feature_size)

# 사전 학습된 가중치 로드
pretrained_model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.roberta.load_state_dict(pretrained_model.state_dict(), strict=False)

# 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


CodeBERTForVulnerabilityDetectionWithFeatures(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


In [22]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,   # batch size for training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="steps",     # 평가 전략을 "steps"로 설정
    eval_steps=500,                   # 500 스텝마다 평가
    fp16=True                        # 16비트 연산 사용
)

# 데이터셋을 위한 데이터 콜레이터 정의
def data_collator(features):
    input_ids = torch.stack([f['input_ids'] for f in features])
    attention_mask = torch.stack([f['attention_mask'] for f in features])
    code_y_input_ids = torch.stack([f['code_y_input_ids'] for f in features])
    code_y_attention_mask = torch.stack([f['code_y_attention_mask'] for f in features])
    labels = torch.stack([f['labels'] for f in features])
    additional_features = torch.stack([f['features'] for f in features])
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'code_y_input_ids': code_y_input_ids,
        'code_y_attention_mask': code_y_attention_mask,
        'features': additional_features,
        'labels': labels
    }

# Trainer 초기화
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    data_collator=data_collator,         # data collator for batching
    compute_metrics=compute_metrics      # 커스텀 메트릭스 함수
)

import torch
torch.cuda.empty_cache()

# 모델 훈련
trainer.train()

# 모델 평가
results = trainer.evaluate()
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8
500,1.694400,1.136932,0.621441,0.621441,0.620680,0.624111,0.621441,0.959119,0.479730,0.442344,0.659884,0.562118,0.622433,0.601093,0.612440,0.677840
1000,0.777000,0.620110,0.808040,0.808040,0.808431,0.809938,0.808040,0.965409,0.695925,0.780362,0.817143,0.764350,0.782609,0.755725,0.849180,0.878740
1500,0.409200,0.412145,0.894472,0.894472,0.894709,0.895637,0.894472,0.971963,0.805430,0.908322,0.869436,0.826025,0.929853,0.908824,0.921875,0.919003
2000,0.211600,0.401720,0.909883,0.909883,0.910496,0.912039,0.909883,0.973479,0.792350,0.898305,0.893491,0.885449,0.955039,0.924012,0.939535,0.946688
2500,0.153700,0.425047,0.919263,0.919263,0.919299,0.920598,0.919263,0.971963,0.821270,0.903567,0.905605,0.902549,0.959350,0.933135,0.957929,0.930016
3000,0.115200,0.367059,0.931658,0.931658,0.931819,0.933165,0.931658,0.967643,0.843165,0.918478,0.917271,0.914027,0.958199,0.937685,0.972713,0.969305
3500,0.058500,0.370202,0.937018,0.937018,0.936732,0.938056,0.937018,0.978328,0.850370,0.929155,0.906205,0.912121,0.965625,0.956391,0.971338,0.972973
4000,0.065000,0.383486,0.942714,0.942714,0.942382,0.943558,0.942714,0.975232,0.855007,0.938608,0.922207,0.936037,0.966929,0.947214,0.970079,0.980707
4500,0.034700,0.346432,0.951424,0.951424,0.951522,0.952548,0.951424,0.978328,0.880682,0.946927,0.927954,0.938650,0.981013,0.967066,0.977707,0.974603
5000,0.026400,0.385727,0.945729,0.945729,0.945552,0.946595,0.945729,0.973643,0.867257,0.950954,0.907010,0.935385,0.977848,0.962853,0.976153,0.968254


{'eval_loss': 0.3829655945301056, 'eval_accuracy': 0.9527638190954774, 'eval_recall_weighted': 0.9527638190954774, 'eval_f1_weighted': 0.9527242930249145, 'eval_f1_macro': 0.9535954009039617, 'eval_f1_micro': 0.9527638190954774, 'eval_f1_class_0': 0.9767441860465116, 'eval_f1_class_1': 0.893371757925072, 'eval_f1_class_2': 0.9466484268125855, 'eval_f1_class_3': 0.9288824383164006, 'eval_f1_class_4': 0.941358024691358, 'eval_f1_class_5': 0.9763779527559054, 'eval_f1_class_6': 0.9773755656108598, 'eval_f1_class_7': 0.9777070063694268, 'eval_f1_class_8': 0.9638932496075353, 'eval_runtime': 51.4265, 'eval_samples_per_second': 58.044, 'eval_steps_per_second': 1.828, 'epoch': 10.0}
